# Video Message Generator from Youtube

Hello, today I will explain how to create an interesting application to create video messages from Youtube Videos.



In SageMaker Notebook we have loaded our custom python enviroment

In [1]:
import sys ,os
print(sys.prefix)

/home/ec2-user/anaconda3/envs/VideoMessage


However if we want to install to our enviroment by using our terminal we have to be careful, becase Sagemaker runs on their own container,
then in the  terminal of Sagemaker we have to load propertly the enviroment that we have created

In [2]:
Sagemaker = True
if Sagemaker :
    env='source activate python3 && conda activate VideoMessage &&'
else:
    env='' 

In [3]:
!echo -e 'import sys , os\nprint(sys.prefix)\nprint("Hello World!")\n'> hello.py

In [4]:
cat hello.py

import sys , os
print(sys.prefix)
print("Hello World!")



For example if we run the hello world without loading we get

In [5]:
os.system('python hello.py')

/home/ec2-user/anaconda3/envs/JupyterSystemEnv
Hello World!


0

As you see in Sagemaker the enviroment is different as we wanted

In [6]:
os.system('source activate python3 && python hello.py')

/home/ec2-user/anaconda3/envs/python3
Hello World!


0

In [7]:
base='source activate python3 &&'

So we need to call with our custom enviroment

In [8]:
os.system('{}python hello.py'.format(env))

/home/ec2-user/anaconda3/envs/VideoMessage
Hello World!


0

In [9]:
#os.system('conda activate VideoMessage && python hello.py')

## Step 1. Setup of the dependencies

In this part we have install all dependencies of our custom program

In [10]:
is_first_time = False

In [11]:
#Install dependency
# Download pretrained model
# Import the os module
import os
# Get the current working directory
parent_dir = os.getcwd()
print(parent_dir)
if is_first_time:
    # Directory 
    directory = "sample_data"
    # Path 
    path = os.path.join(parent_dir, directory) 
    print(path)
    try:
        os.mkdir(path)
        print("Directory '% s' created" % directory) 
    except Exception:
         print("Directory '% s'was already created" % directory)
    os.system('git clone https://github.com/Cloud-Data-Science/Wav2Lip')
    os.system('cd Wav2Lip &&{} pip install  -r requirements.txt'.format(env))

/home/ec2-user/SageMaker/VideoMessageGen


In [12]:
from utils.default_models import ensure_default_models
from pathlib import Path

In [13]:
## Load the models one by one.
print("Preparing the models of Wav2Lip")
ensure_default_models(Path("Wav2Lip"))

Preparing the models of Wav2Lip
Wav2Lip/checkpoints/wav2lip_gan.pth
Wav2Lip/face_detection/detection/sfd/s3fd.pth


In [14]:
if is_first_time:
    os.system('git clone https://github.com/Cloud-Data-Science/Coqui-TTS -b multilingual-torchaudio-SE TTS')
    os.system('{} pip install -q -e TTS/'.format(env))
    os.system('{} pip install -q torchaudio==0.9.0'.format(env))
    os.system('{} pip install -q youtube-dl'.format(env))
    os.system('{} pip install ffmpeg-python'.format(env))
    os.system('{} pip install gradio==3.0.4'.format(env))

In [15]:
#this code for recording audio
from IPython.display import HTML, Audio
#from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

In [16]:
from IPython.display import HTML
from base64 import b64encode
def showVideo(path):
  mp4 = open(str(path),'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width=700 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url)

from IPython.display import clear_output

In [17]:
from subprocess import call
import os

In [18]:
URL = 'https://www.youtube.com/watch?v=xw5dvItD5zY'

In [19]:
def youtube_trim(url,start,end):
    # Get the current working directory
    parent_dir = os.getcwd()
    YOUTUBE_URL=url
    #Select a Youtube Video
    #find youtube video id
    from urllib import parse as urlparse
    url_data = urlparse.urlparse(YOUTUBE_URL)
    query = urlparse.parse_qs(url_data.query)
    YOUTUBE_ID = query["v"][0]
    url_download ="https://www.youtube.com/watch?v={}".format(YOUTUBE_ID)
    print(url_download)
    # Trim the video (start, end) seconds
    start =  start
    end =  end
    #Note: the trimmed video must have face on all frames
    interval = end - start
    #call(["rm","-df","youtube.mp4"])
    # download the youtube with the given ID
    #call(["youtube-dl", "-f","bestvideo[ext=mp4]","--output","youtube.%(ext)s",url_download])
    #call(["youtube-dl", "-f","mp4","--output","youtube.%(ext)s",url_download])
    os.system('{} youtube-dl -f  mp4 --output youtube.%(ext)s {} '.format(env,url_download))
    #!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID
    trimmed_video= parent_dir+'/sample_data/input_vid.mp4'
    trimmed_audio= parent_dir+'/sample_data/input_audio.mp3'
    #delete video.mp4 if already exits
    call(["rm","-f",trimmed_video])
    call(["rm","-f",trimmed_audio])
    #!rm -f {trimmed_video}
    # cut the video                 
    call(["ffmpeg","-y","-i","youtube.mp4","-ss", str(start),"-t",str(interval),"-async","1",trimmed_video])
    #!ffmpeg -y -i youtube.mp4 -ss {start} -t {interval} -async 1 {trimmed_video}
    # cut the audio
    call(["ffmpeg","-i",trimmed_video, "-q:a", "0", "-map","a",trimmed_audio])
    #Preview trimmed video
    #clear_output()
    print("Trimmed Video+Audio")
    return trimmed_video, trimmed_audio
    

In [22]:
trimmed_video, trimmed_audio=youtube_trim(URL,1,10)
showVideo(trimmed_video)

https://www.youtube.com/watch?v=xw5dvItD5zY


sh: -c: line 0: syntax error near unexpected token `('
sh: -c: line 0: `source activate python3 && conda activate VideoMessage && youtube-dl -f  mp4 --output youtube.%(ext)s https://www.youtube.com/watch?v=xw5dvItD5zY '
ffmpeg version 5.1.1-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --enable-libvidstab --enable-libvo-amrwbenc --enable-libvpx --enable-libweb

Trimmed Video+Audio


frame=  225 fps= 71 q=-1.0 Lsize=     495kB time=00:00:09.00 bitrate= 450.5kbits/s speed=2.82x    
video:346kB audio:141kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 1.824652%
[libx264 @ 0x73e4dc0] frame I:1     Avg QP:15.93  size: 40929
[libx264 @ 0x73e4dc0] frame P:57    Avg QP:19.77  size:  3605
[libx264 @ 0x73e4dc0] frame B:167   Avg QP:26.13  size:   639
[libx264 @ 0x73e4dc0] consecutive B-frames:  0.4%  0.9%  2.7% 96.0%
[libx264 @ 0x73e4dc0] mb I  I16..4: 16.0% 69.8% 14.1%
[libx264 @ 0x73e4dc0] mb P  I16..4:  0.5%  1.9%  0.2%  P16..4: 13.8%  3.8%  1.8%  0.0%  0.0%    skip:78.0%
[libx264 @ 0x73e4dc0] mb B  I16..4:  0.0%  0.0%  0.0%  B16..8: 12.4%  0.4%  0.0%  direct: 0.0%  skip:87.2%  L0:46.2% L1:52.5% BI: 1.4%
[libx264 @ 0x73e4dc0] 8x8 transform intra:71.8% inter:82.9%
[libx264 @ 0x73e4dc0] coded y,uvDC,uvAC intra: 59.1% 57.7% 26.4% inter: 1.5% 1.2% 0.0%
[libx264 @ 0x73e4dc0] i16 v,h,dc,p: 39% 15% 17% 29%
[libx264 @ 0x73e4dc0] i8 v,h,dc,ddl,ddr,vr,hd,vl,hu

In [20]:
if is_first_time:
    os.system('git clone https://github.com/Cloud-Data-Science/Voice-Cloning VOICE')
    os.system('cd VOICE &&{} pip install  -r requirements.txt'.format(env))
    os.system('mv ./VOICE/* .')

## We verify the pandas and numpy versions

In [21]:
print("In our enviroment")
os.system('{} pip show pandas numpy'.format(env))

In our enviroment
Name: pandas
Version: 1.3.5
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: /home/ec2-user/anaconda3/envs/VideoMessage/lib/python3.7/site-packages
Requires: numpy, python-dateutil, pytz
Required-by: gradio, TTS
---
Name: numpy
Version: 1.19.5
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD
Location: /home/ec2-user/anaconda3/envs/VideoMessage/lib/python3.7/site-packages
Requires: 
Required-by: gradio, librosa, matplotlib, numba, opencv-contrib-python, opencv-contrib-python-headless, opencv-python, pandas, resampy, scikit-learn, scipy, tensorboardX, torchvision, TTS, umap-learn


0

In [22]:
print("In the base enviroment")
os.system('{} pip show pandas numpy'.format(base))

In the base enviroment
Name: pandas
Version: 1.3.4
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: /home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages
Requires: numpy, python-dateutil, pytz
Required-by: autovizwidget, hdijupyterutils, sagemaker, seaborn, shap, smclarify, sparkmagic, statsmodels
---
Name: numpy
Version: 1.20.3
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD
Location: /home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages
Requires: 
Required-by: astropy, bkcharts, bokeh, Bottleneck, daal4py, h5py, hdijupyterutils, imagecodecs, imageio, matplotlib, mkl-fft, mkl-random, numba, numexpr, opencv-python, pandas, patsy, pyarrow, pyerfa, PyWavelets, sagemaker, sag

0

In [27]:
print("In the Jupyter enviroment")
os.system('pip show pandas numpy')

Name: pandas
Version: 1.3.5
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: /home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages
Requires: numpy, python-dateutil, pytz
Required-by: autovizwidget, hdijupyterutils, sagemaker, sparkmagic
---
Name: numpy
Version: 1.21.6
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD
Location: /home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages
Requires: 
Required-by: Bottleneck, hdijupyterutils, numexpr, pandas, sagemaker, sagemaker-pyspark, sparkmagic


0

In [28]:
import numpy
numpy.version.version

'1.19.5'

In [29]:
if is_first_time:
    os.system('{} pip install opencv-contrib-python-headless==4.1.2.30'.format(env))

In [30]:
#pip install numpy==1.19.5 --upgrade --ignore-installed

In [31]:
import sys
VOICE_PATH = "utils/"
# add libraries into environment
sys.path.append(VOICE_PATH) # set this if VOICE is not installed globally

In [32]:
from utils.voice import *

/home/ec2-user/SageMaker/VideoMessageGen
 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:False
 | > do_amp_to_db_mel:True
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Using model: vits
 > Speaker manager is loaded with 6 speakers: female-en-5, female-en-5
, female-pt-4
, male-en-2, male-en-2
, male-pt-3

 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:64
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None

In [33]:
Text=' I am Clonning your Voice , Charles. Machine intelligence is the last invention that humanity will ever need to make.'
Voicetoclone=trimmed_audio
print(Voicetoclone)

/home/ec2-user/SageMaker/VideoMessageGen/sample_data/input_audio.mp3


In [21]:
def create_video(Text,Voicetoclone):
    out_audio=greet(Text,Voicetoclone)
    current_dir=os.getcwd()
    clonned_audio = os.path.join(current_dir, out_audio)
    
    #Start Crunching and Preview Output
    #Note: Only change these, if you have to
    pad_top =  0#@param {type:"integer"}
    pad_bottom =  10#@param {type:"integer"}
    pad_left =  0#@param {type:"integer"}
    pad_right =  0#@param {type:"integer"}
    rescaleFactor =  1#@param {type:"integer"}
    nosmooth = False #@param {type:"boolean"}
    
    if nosmooth == False:
        os.system('{} cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "../sample_data/input_vid.mp4" --audio "../out/clonned_audio.wav" --pads {} {} {} {} --resize_factor {}'.format(env,pad_top ,pad_bottom ,pad_left ,pad_right ,rescaleFactor))
    else:
        os.system('{} cd Wav2Lip && python inference.py --checkpoint_path checkpoints/wav2lip_gan.pth --face "../sample_data/input_vid.mp4" --audio "../out/clonned_audio.wav" --pads {} {} {} {} --resize_factor {} --nosmooth'.format(env,pad_top ,pad_bottom ,pad_left ,pad_right ,rescaleFactor))

    #clear_output()
    print("Creation of Video done")

In [22]:
create_video(Text,Voicetoclone)

path url
/home/ec2-user/SageMaker/VideoMessageGen/sample_data/input_audio.mp3


usage: ffmpeg-normalize [-h] [-o OUTPUT [OUTPUT ...]] [-of OUTPUT_FOLDER] [-f]
                        [-d] [-v] [-q] [-n] [-pr] [--version]
                        [-nt {ebu,rms,peak}] [-t TARGET_LEVEL] [-p]
                        [-lrt LOUDNESS_RANGE_TARGET]
                        [--keep-loudness-range-target] [-tp TRUE_PEAK]
                        [--offset OFFSET] [--dual-mono] [--dynamic]
                        [-c:a AUDIO_CODEC] [-b:a AUDIO_BITRATE]
                        [-ar SAMPLE_RATE] [-koa] [-prf PRE_FILTER]
                        [-pof POST_FILTER] [-vn] [-c:v VIDEO_CODEC] [-sn]
                        [-mn] [-cn] [-ei EXTRA_INPUT_OPTIONS]
                        [-e EXTRA_OUTPUT_OPTIONS] [-ofmt OUTPUT_FORMAT]
                        [-ext EXTENSION]
                        input [input ...]
ffmpeg-normalize: error: argument -o/--output: expected at least one argument
/home/ec2-user/anaconda3/envs/VideoMessage/lib/python3.7/site-packages/librosa/core/audio.py:162: U

 > text:  I am Clonning your Voice , Charles. Machine intelligence is the last invention that humanity will ever need to make.
Generated Audio


 > Saving output to out/clonned_audio.wav


  0%|          | 0/12 [00:02<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]/home/ec2-user/anaconda3/envs/VideoMessage/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)

100%|██████████| 2/2 [00:53<00:00, 26.94s/it]ffmpeg version 5.1.1-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-liba

Using cuda for inference.
Reading video frames...
Number of frames available for inference: 225
(80, 612)
Length of mel chunks: 188
Recovering from OOM error; New batch size: 8
Load checkpoint from: checkpoints/wav2lip_gan.pth
Model loaded
Creation of Video done


In [23]:
#Preview output video
print("Final Video Preview")
final_video= parent_dir+'/Wav2Lip/results/result_voice.mp4'
print("Dowload this video from", final_video)
showVideo(final_video)

Final Video Preview
Dowload this video from /home/ec2-user/SageMaker/VideoMessageGen/Wav2Lip/results/result_voice.mp4


In [36]:
def video_generator(text_to_say,URL,initial_time,final_time):
    time_format_check(initial_time)
    time_format_check(final_time)
    trimmed_video, trimmed_audio=youtube_trim(URL,initial_time,final_time)
    return trimmed_video

In [37]:
import datetime
def time_format_check(input1):
    timeformat = "%H:%M:%S"
    #input1 = input("At what time did sensor 1 actuate? ")
    try:
        validtime = datetime.datetime.strptime(input1, timeformat)
        print("The time format is valid", input1)
        #Do your logic with validtime, which is a valid format
    except ValueError:
        print("The time {}  has not valid format hh:mm:ss".format(input1))

In [40]:
text_to_say=gr.inputs.Textbox(label='What would you like the voice to say? (max. 2000 characters per request)')
url =gr.inputs.Textbox(label = "Enter the YouTube URL below:")
initial_time = gr.inputs.Textbox(label='Initial time of trim? (format: hh:mm:ss)')
final_time= gr.inputs.Textbox(label='Final time to trim? (format: hh:mm:ss)')
demo = gr.Interface(fn = video_generator,
            inputs = [text_to_say,url,initial_time,final_time],
            outputs = 'video', 
            verbose = True,
            title = 'Video Speech Generator from Youtube Videos',
            description = 'A simple application that replaces the original speech of the video by your text. Wait one minute to process.',
            article = 
                        '''<div>
                            <p style="text-align: center"> 
                            All you need to do is to paste the Youtube link and 
                            set the initial time and final time of the real speach.
                            (The limit of the trim is 2 minutes)
                            hit submit, then wait for compiling.
                            After that click on Play/Pause for listing to the video. 
                            The video is saved in an mp4 format.
                             For more information visit <a href="https://ruslanmv.com/">ruslanmv.com</a>
                            </p>
                        </div>''',

           examples = [
                        ['I am clonning your voice, Charles!. Machine intelligence is the last invention that humanity will ever need to make.',
                           "https://www.youtube.com/watch?v=xw5dvItD5zY", "00:00:01","00:00:10"],
                        ]           
            )

/home/ec2-user/anaconda3/envs/VideoMessage/lib/python3.7/site-packages/gradio/deprecation.py:40: UserWarning: `verbose` parameter is deprecated, and it has no effect
  warnings.warn(value)


In [41]:
demo.launch()

Running on local URL:  http://127.0.0.1:7860/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7f329e5fcc10>, 'http://127.0.0.1:7860/', None)

The time format is valid 00:00:01
The time format is valid 00:00:10
https://www.youtube.com/watch?v=xw5dvItD5zY


Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/VideoMessage/lib/python3.7/site-packages/gradio/routes.py", line 275, in predict
    output = await app.blocks.process_api(body, username, session_state)
  File "/home/ec2-user/anaconda3/envs/VideoMessage/lib/python3.7/site-packages/gradio/blocks.py", line 274, in process_api
    predictions = await run_in_threadpool(block_fn.fn, *processed_input)
  File "/home/ec2-user/anaconda3/envs/VideoMessage/lib/python3.7/site-packages/starlette/concurrency.py", line 41, in run_in_threadpool
    return await anyio.to_thread.run_sync(func, *args)
  File "/home/ec2-user/anaconda3/envs/VideoMessage/lib/python3.7/site-packages/anyio/to_thread.py", line 32, in run_sync
    func, *args, cancellable=cancellable, limiter=limiter
  File "/home/ec2-user/anaconda3/envs/VideoMessage/lib/python3.7/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/home/ec2-user/anaconda3/en